<a href="https://colab.research.google.com/github/hfelizzola/Investigaciones-de-Operaciones-I/blob/main/flujo-redes/03_modelo_transporte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo de Transporte

In [ ]:
!pip install gamspy --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 3.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from gamspy import Container, Set, Variable, Parameter, Equation, Model, Sense, Sum, Options

In [ ]:
m = Container()

# Sets
planta = Set(m, name='planta', records=['P1','P2','P3'])
almacen = Set(m, name='almacen', records=['A1','A2','A3','A4'])

# Variable
x = Variable(m, name='x', type='Positive', domain=[planta,almacen])

# Parameter
capacidad = Parameter(m, name='capacidad', domain=[planta], records=np.array([75,125,100]))
demanda = Parameter(m, name='demanda', domain=[almacen], records=np.array([80,65,70,85]))
costo = Parameter(m, name='costo', domain=[planta,almacen], records=np.array([[464,513,654,867],
                                                                              [352,416,690,791],
                                                                              [995,682,388,685]]))

# Equations
capacidad_max = Equation(m, name='capacidad_max', domain=[planta])
capacidad_max[planta] = Sum(almacen, x[planta,almacen]) == capacidad[planta]

demanda_max = Equation(m, name='demanda_max', domain=[almacen])
demanda_max[almacen] = Sum(planta, x[planta,almacen]) == demanda[almacen]

costo_total = Sum([planta,almacen], costo[planta,almacen] * x[planta,almacen])

# Modelo
modelo_transporte = Model(m, name='modelo_transporte', objective=costo_total, sense=Sense.MIN, problem='LP', equations=[capacidad_max, demanda_max])
modelo_transporte.solve()

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,152535.0,8,13,LP,CPLEX,0.0


In [ ]:
x.records.pivot(index="planta", columns="almacen", values='level')

almacen,A1,A2,A3,A4
planta,,,,
P1,0.0,20.0,0.0,55.0
P2,80.0,45.0,0.0,0.0
P3,0.0,0.0,70.0,30.0


In [ ]:
capacidad_max.records

,planta,level,marginal,lower,upper,scale
0,P1,75.0,97.0,75.0,75.0,1.0
1,P2,125.0,0.0,125.0,125.0,1.0
2,P3,100.0,-85.0,100.0,100.0,1.0


In [ ]:
demanda_max.records

,almacen,level,marginal,lower,upper,scale
0,A1,80.0,352.0,80.0,80.0,1.0
1,A2,65.0,416.0,65.0,65.0,1.0
2,A3,70.0,473.0,70.0,70.0,1.0
3,A4,85.0,770.0,85.0,85.0,1.0
